In [1]:
import findspark
findspark.init()

In [6]:
from pyspark.sql import SparkSession

sc = SparkSession.builder.appName("PysparkExample")\
.config ("spark.sql.shuffle.partitions", "50")\
.config("spark.driver.maxResultSize","5g")\
.config ("spark.sql.execution.arrow.enabled", "true")\
.getOrCreate()

## 1

In [23]:
sc.sql("drop table if exists mst_users")

DataFrame[]

In [24]:
sc.sql(
"CREATE TABLE mst_users(\
    user_id         varchar(255)\
  , register_date   varchar(255)\
  , register_device integer\
)"
)

DataFrame[]

In [25]:
sc.sql("select * from mst_users").show()

+-------+-------------+---------------+
|user_id|register_date|register_device|
+-------+-------------+---------------+
+-------+-------------+---------------+



In [28]:
sc.sql("INSERT INTO mst_users VALUES\
('U001', '2016-08-26', 1)\
,('U002', '2016-08-26', 2)\
,('U003', '2016-08-27', 3)")

DataFrame[]

In [29]:
sc.sql("select * from mst_users").show()

+-------+-------------+---------------+
|user_id|register_date|register_device|
+-------+-------------+---------------+
|   U001|   2016-08-26|              1|
|   U002|   2016-08-26|              2|
|   U003|   2016-08-27|              3|
+-------+-------------+---------------+



In [47]:
sc.sql(
    "SELECT user_id,\
    CASE when register_device = 1 then '데스크톱'\
    when register_device = 2 then '스마트폰'\
    when register_device = 3 then '애플리케이션'\
    END AS device_name FROM mst_users").show()

+-------+------------+
|user_id| device_name|
+-------+------------+
|   U001|    데스크톱|
|   U002|    스마트폰|
|   U003|애플리케이션|
+-------+------------+



## 2

In [49]:
sc.sql(
"create table access_log(\
stamp varchar(255),\
referrer string,\
url string)")

DataFrame[]

In [51]:
sc.sql(
"insert into access_log values\
('2016-08-26 12:02:00', 'http://www.other.com/path1/index.php?k1=v1&k2=v2#Ref1',\
'http://www.example.com/video/detail?id=001')\
,('2016-08-26 12:02:01', 'http://www.other.net/path1/index.php?k1=v1&k2=v2#Ref1',\
'http://www.example.com/video#ref')\
,('2016-08-26 12:02:01', 'https://www.other.com/',\
'http://www.example.com/book/detail?id=002')")

DataFrame[]

In [65]:
sc.sql("select * from access_log").show()

+-------------------+--------------------+--------------------+
|              stamp|            referrer|                 url|
+-------------------+--------------------+--------------------+
|2016-08-26 12:02:00|http://www.other....|http://www.exampl...|
|2016-08-26 12:02:01|http://www.other....|http://www.exampl...|
|2016-08-26 12:02:01|https://www.other...|http://www.exampl...|
+-------------------+--------------------+--------------------+



In [55]:
sc.sql(
"select stamp\
, parse_url(referrer,'HOST') as referrer_host\
 from access_log").show()

+-------------------+-------------+
|              stamp|referrer_host|
+-------------------+-------------+
|2016-08-26 12:02:00|www.other.com|
|2016-08-26 12:02:01|www.other.net|
|2016-08-26 12:02:01|www.other.com|
+-------------------+-------------+



In [58]:
sc.sql(
"select\
 stamp\
 ,url\
 ,parse_url(url,'PATH') as path\
 ,parse_url(url,'QUERY','id') as id\
  from access_log").show()

+-------------------+--------------------+-------------+----+
|              stamp|                 url|         path|  id|
+-------------------+--------------------+-------------+----+
|2016-08-26 12:02:00|http://www.exampl...|/video/detail| 001|
|2016-08-26 12:02:01|http://www.exampl...|       /video|null|
|2016-08-26 12:02:01|http://www.exampl...| /book/detail| 002|
+-------------------+--------------------+-------------+----+



## 3

In [59]:
sc.sql(
"select \
stamp\
,url\
,split(parse_url(url, 'PATH'),'/')[1] as path1\
,split(parse_url(url, 'PATH'),'/')[2] as path2 \
from access_log").show()

+-------------------+--------------------+-----+------+
|              stamp|                 url|path1| path2|
+-------------------+--------------------+-----+------+
|2016-08-26 12:02:00|http://www.exampl...|video|detail|
|2016-08-26 12:02:01|http://www.exampl...|video|  null|
|2016-08-26 12:02:01|http://www.exampl...| book|detail|
+-------------------+--------------------+-----+------+



## 4

In [61]:
sc.sql(
"select \
current_date as dt\
,current_timestamp as stamp").show()

+----------+--------------------+
|        dt|               stamp|
+----------+--------------------+
|2020-04-08|2020-04-08 21:26:...|
+----------+--------------------+



In [66]:
sc.sql(
"select \
cast('2020-01-04' as date) as dt\
,timestamp '2020-04-08 09:33:00' as stamp").show()

+----------+-------------------+
|        dt|              stamp|
+----------+-------------------+
|2020-01-04|2020-04-08 09:33:00|
+----------+-------------------+



In [69]:
sc.sql(
"select \
stamp\
,year(stamp) as year\
,month(stamp) as month\
,day(stamp) as day\
,hour(stamp) as hour \
from\
(select cast('2020-04-08 21:36:23' as timestamp) as stamp) as t").show()

+-------------------+----+-----+---+----+
|              stamp|year|month|day|hour|
+-------------------+----+-----+---+----+
|2020-04-08 21:36:23|2020|    4|  8|  21|
+-------------------+----+-----+---+----+



In [71]:
sc.sql(
"select \
stamp\
,substring(stamp,1,4) as year\
,substring(stamp,6,2) as month\
,substring(stamp,9,2) as day\
,substring(stamp,12,2) as hour\
,substring(stamp,1,7) as year_month \
from\
(select cast('2020-04-08 21:41:23' as string) as stamp)").show()

+-------------------+----+-----+---+----+----------+
|              stamp|year|month|day|hour|year_month|
+-------------------+----+-----+---+----+----------+
|2020-04-08 21:41:23|2020|   04| 08|  21|   2020-04|
+-------------------+----+-----+---+----+----------+



## 5

In [72]:
sc.sql(
"create table purchase_log_with_coupon (\
Purchase_id varchar(255)\
,amount integer\
,coupon integer)")

DataFrame[]

In [75]:
sc.sql(
"insert into purchase_log_with_coupon values\
('10001',3280,null)\
,('10002',4650,500)\
,('10003',3870,null)")

DataFrame[]

In [76]:
sc.sql(
"select * from purchase_log_with_coupon").show()

+-----------+------+------+
|Purchase_id|amount|coupon|
+-----------+------+------+
|      10001|  3280|  null|
|      10002|  4650|   500|
|      10003|  3870|  null|
+-----------+------+------+



In [77]:
sc.sql(
"select \
Purchase_id\
, amount - coupon as discount_amount1\
, amount - coalesce(coupon,0) as discount_amount2 \
from purchase_log_with_coupon").show()

+-----------+----------------+----------------+
|Purchase_id|discount_amount1|discount_amount2|
+-----------+----------------+----------------+
|      10001|            null|            3280|
|      10002|            4150|            4150|
|      10003|            null|            3870|
+-----------+----------------+----------------+

